In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as f

import torchvision.transforms.v2 as T
import albumentations as A
from albumentations.core.transforms_interface import DualTransform
from torchvision.utils import save_image

import pydicom as dcm
import cv2

from tqdm import tqdm

## Helper functions

In [2]:
def set_device():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    return device    

def set_data_paths():
    data_path = {}
    data_path['root'] = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
    data_path['train'] = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
    data_path['segmentations'] = '/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations'
    data_path['test'] = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
    data_path['masks'] = '/kaggle/working/mask_slices'
    return data_path

def create_segmentation_dict():
    # Mask organ label mapping
    segmentation_dict = {}
    segmentation_dict['segmentation_class_to_inx'] = {'Background': 0, 'Liver': 1, 'Spleen': 2, 'Kidney_left': 3, 'Kidney_right': 4, 'Bowel': 5}
    segmentation_dict['segmentaiton_inx_to_class'] = {0: 'Background', 1: 'Liver', 2:'Spleen', 3:'Kidney_left', 4:'Kidney_right', 5 :'Bowel'}
    segmentation_dict['final_output'] = ['Background', 'Bowel', 'Kidney' , 'Liver' , 'Spleen']
    return segmentation_dict

def standardize_pixel_array(dcm: dcm.dataset.FileDataset) -> np.ndarray:
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
    return pixel_array

def load_CT_slice(filepath, downsample_rate=1):
    ds = dcm.dcmread(filepath)
    image = standardize_pixel_array(ds)

    # find rescale params
    if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
        intercept = float(ds.RescaleIntercept)
        slope = float(ds.RescaleSlope)

    # find clipping params
    center = int(ds.WindowCenter)
    width = int(ds.WindowWidth)
    low = center - width / 2
    high = center + width / 2    

    image = (image * slope) + intercept
    image = np.clip(image, low, high)
    image = image - image.min()
    if image.max() > 0:
        image = (image / image.max() ).astype(np.float32)
    image = image[::downsample_rate, ::downsample_rate]
    image = np.array(image * 255).astype(np.uint8)
    return image

def sorted_dcm_labels(folder):
    try:
        filenames = os.listdir(folder)
        names = sorted( [int(filename.split('.')[0]) for filename in filenames] )
        return names
    except:
        return []

def standardize_pixel_array(dcm: dcm.dataset.FileDataset) -> np.ndarray:
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
    return pixel_array

def load_CT_slice(filepath, downsample_rate=1):
    ds = dcm.dcmread(filepath)
    image = standardize_pixel_array(ds)

    # find rescale params
    if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
        intercept = float(ds.RescaleIntercept)
        slope = float(ds.RescaleSlope)

    # find clipping params
    center = int(ds.WindowCenter)
    width = int(ds.WindowWidth)
    low = center - width / 2
    high = center + width / 2    

    image = (image * slope) + intercept
    image = np.clip(image, low, high)
    image = image - image.min()
    if image.max() > 0:
        image = (image / image.max() ).astype(np.float32)
    image = image[::downsample_rate, ::downsample_rate]
    image = np.array(image * 255).astype(np.uint8)
    return image

def get_z_acquisition_direction(session_path):
    instances = sorted_dcm_labels(session_path)
    
    # extract z_start
    instance_start = instances[0]
    filepath = session_path + '/' + str(instance_start) + '.dcm'
    z_start = dcm.dcmread( filepath ).ImagePositionPatient[-1]
    # extract z_end
    instance_end = instances[-1]
    filepath = session_path + '/' + str(instance_end) + '.dcm'
    z_end = dcm.dcmread( filepath ).ImagePositionPatient[-1]
        
    if z_end < z_start:
        return 'downward'
    else:
        return 'upward'

def CT_remove_distractions(img):
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8U)

    thresh = cv2.threshold(img, 2, 255, cv2.THRESH_BINARY)[1]
    output = cv2.connectedComponentsWithStats(thresh, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output
    components = {label: (labels == label).sum() for label in sorted(np.unique(labels))}

    good_components = np.argsort(list(components.values()))[::-1][:2]
    good_areas = np.isin(labels, good_components)

    img_clean = img * good_areas
    return img_clean

def CT_crop_background(img):
    i, j = np.where(img > 5)
    i_min = i.min()
    i_max = i.max()
    j_min = j.min()
    j_max = j.max()
    
    img_cropped = img[i_min:i_max, j_min:j_max]
    return img_cropped, (i_min, i_max, j_min, j_max)

def prepare_layered_mask(file_path, segmentation_dict):
    mask = cv2.imread(file_path)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    masks_dict = one_hot(mask, classes=segmentation_dict['segmentation_class_to_inx'])
    masks_dict['Kidney'] = masks_dict['Kidney_right'] | masks_dict['Kidney_left']
    del masks_dict['Kidney_right'], masks_dict['Kidney_left']

    masks_dict_sorted = {key: masks_dict[key] for key in segmentation_dict['final_output']}
    masks = [np.array(mask).astype(np.float32) for mask in masks_dict_sorted.values()]
    return masks


class Clean_CT(DualTransform):

    def __init__(self, image_size=None, always_apply=True, p=1.0):
        super().__init__(always_apply, p)
        self.borders = None
        self.image_size = image_size
        
    def apply(self, img, **params):
        img = img.copy()
        img = np.array(img)
        img_clean = CT_remove_distractions(img)
        img_clean_cropped, self.borders = CT_crop_background(img_clean)
        img_clean_cropped_resized = A.Compose([
            A.Resize(int(self.image_size[1]*0.7), self.image_size[1] - 10, interpolation=cv2.INTER_NEAREST_EXACT),
            A.PadIfNeeded(min_height=self.image_size[0], min_width=self.image_size[1], p=1, border_mode=cv2.BORDER_CONSTANT, value=0)
        ])(image=img_clean_cropped)['image']
        return img_clean_cropped_resized
    
    def apply_to_mask(self, mask, fill_value=0, **params):
        mask = mask.copy()
        mask = np.array(mask)
        mask = mask[self.borders[0]:self.borders[1], self.borders[2]:self.borders[3]]
        mask = A.Compose([
            A.Resize(int(self.image_size[1]*0.7), self.image_size[1] - 10, interpolation=cv2.INTER_NEAREST_EXACT),
            A.PadIfNeeded(min_height=self.image_size[0], min_width=self.image_size[1], p=1, border_mode=cv2.BORDER_CONSTANT, value=fill_value)
        ])(image=mask)['image']
        return mask
    
    def apply_to_masks(self, masks, **params):
        out = []
        for ind, mask in enumerate(masks):
            if ind == 0: # background
                val = 1.0
            else: 
                val = 0.0
            out.append( self.apply_to_mask(mask, fill_value=val, **params) )
        return out

# ________________________________________________________________________________________________________________ DataFrame tools
def create_seg_slices(data_path, SAVE_ROOT, downsample_rate=1):
    df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')

    files = sorted([int(filename.split('.')[0]) for filename in os.listdir(data_path['segmentations'])])
    df = df.loc[df['series_id'].isin(files), :]

    df['mask_path'] = df.apply( lambda row: os.path.join(SAVE_ROOT, str(int(row['patient_id'])), str(int(row['series_id'])) ), axis=1)
    df['seg_path'] = df.apply( lambda row: os.path.join(data_path['segmentations'], str(int(row['series_id'])) + '.nii' ), axis=1)    
    
    for ind in tqdm(range( df.shape[0] )):
        row = df.iloc[ind, :]
        subj = str(int(row['patient_id']))
        series_id = str(int(row['series_id']))
        seg_path = row['seg_path']

        try:
            os.makedirs( row['mask_path'] )
        except:
            pass

        sorted_filenames = sorted_dcm_labels( os.path.join(data_path['train'], subj, series_id) )[::downsample_rate]
        sorted_filenames_path = [os.path.join(row['mask_path'], str(sorted_filename) + '.png') for sorted_filename in sorted_filenames]

        # load mask of this session
        mask = create_3D_segmentations(seg_path)

        for ind, filepath  in enumerate(sorted_filenames_path):
            mask_slice = np.squeeze( mask[ind, :, :] )

            # save file
            cv2.imwrite(filepath, mask_slice)

        if ind % 10 == 0:
            gc.collect() 
    
def create_patient_list(data_path, downsample_rate=1, mode='train'):
    df = pd.read_csv( os.path.join('/kaggle/input/rsna-2023-abdominal-trauma-detection', mode + '_series_meta.csv') )

    unique_aortic_hu_list = df.groupby('patient_id')['aortic_hu'].apply(lambda x: list(np.unique(x)))
    df['aortic_hu_values'] = df['patient_id'].apply(lambda x: unique_aortic_hu_list.loc[x])

    files = sorted([int(filename.split('.')[0]) for filename in os.listdir(data_path['segmentations'])])
    df['has_seg'] = df['series_id'].isin(files).astype('category')

    df['instance_number'] = df.apply(lambda row: 
                                                [slice_number for slice_number in 
                                                 sorted_dcm_labels( os.path.join(data_path[mode], str(int(row['patient_id'])), str(int(row['series_id'])) ) )[::downsample_rate] ], axis=1)
    
    df['DS_RATE'] = downsample_rate
    df['DS_RATE'] = df['DS_RATE'].astype('category')

    
    # EXPLODE OVER INSTANC_NUMBER COLUMN
    df = df.explode(['instance_number'])
    df = df.loc[ df['instance_number'].notnull(), :]    
    df['instance_number'] = df['instance_number'].astype(np.uint32)

    
    # load CNN_targets and merge
    if mode == 'train':
        # FIND OUT WHETHER EACH SLICE IS MARKED BY BOWEL OR EXTRAVASATION INJURY
        image_level_labels = pd.read_csv( os.path.join(data_path['root'], 'image_level_labels.csv') ) 
        df = df.merge(image_level_labels, on=['patient_id','series_id', 'instance_number'], how='left')    
        df['bowel_injury_marker'] = (df['injury_name'] == 'Bowel').astype('bool')
        df['extravasation_injury_marker'] = (df['injury_name'] == 'Active_Extravasation').astype('bool')
        
        CNN_targets = pd.read_csv( os.path.join(data_path['root'], 'train.csv') )
        for col in CNN_targets.columns:
            if col == 'patient_id':
                continue
            CNN_targets[col] = CNN_targets[col].astype('bool')
            
        df = df.merge(CNN_targets, on='patient_id', how='left')
    
    df = df.sort_values(['patient_id', 'series_id','instance_number'])
    return df

## Configurations

In [3]:
class CFG():
    def __init__(self):
        self.device = set_device()
        self.data_path = set_data_paths()
        self.data_path['masks'] = '/kaggle/input/rsna-masks'
        self.segmentation_dict = create_segmentation_dict()
        self.num_slices = 30
        
        self.create_masks = 0
        self.create_patient_list = 0
        
        self.DS_RATE = 1 # downsamples slices in each session
        self.image_size = (256, 256)
        self.validation_size = 0.1
                
        # unet
        self.UNet = {}
        self.UNet['channel_list'] = [1, 32, 64, 128, 256]
        
        # image transforms
        self.transform_dict = {}
        self.transform_dict['basic'] = A.Compose([
            A.Equalize(p=1),
            Clean_CT(image_size=self.image_size),
        ])
        mean = torch.tensor(0.2365)#.to(self.device)
        std = torch.tensor(0.3205)#.to(self.device)
        self.transform_dict['normalize'] = A.Normalize(mean=mean, std=std)

cfg = CFG()
np.set_printoptions(precision=2,suppress=True)

cpu


## Define UNET

In [4]:
class unet(nn.Module):
    def __init__(self, CHANNEL_LIST, OUTPUT_CHANNEL=1):
        super().__init__()
        self.encoder_block = encoder(CHANNEL_LIST[:-1]) # 128 channels
        self.flat_block = conv_block(CHANNEL_LIST[-2], CHANNEL_LIST[-1]) # 256 channels
        self.decoder_block = decoder(CHANNEL_LIST[::-1][:-1]) # 16 channel
        self.output_block = conv_block(CHANNEL_LIST[1], OUTPUT_CHANNEL) # 1 channel
        
    def forward(self, x):
        encoder_outputs, x = self.encoder_block(x)
        x = self.flat_block(x)
        x = self.decoder_block(x, encoder_outputs)
        x = f.softmax( self.output_block(x), dim=1)
        return x

class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 7, padding='same', bias=False)
        self.batchnorm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 7, padding='same')

    def forward(self, x):
        x = f.relu( self.batchnorm1( self.conv1(x) ) )
        x = self.conv2(x)
        return x

class encoder(nn.Module):
    def __init__(self, channel_list):
        super().__init__()
        self.blocks = nn.ModuleList([
            conv_block(channel_list[i], channel_list[i+1])
            for i in range(len(channel_list)-1)
        ])
        self.maxpool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(0.1)
        
    def forward(self, x):
        encoder_outputs = []
        for block in self.blocks:
            x = block(x)
            encoder_outputs.append(x)
            x = self.maxpool(x)
            x = self.dropout(x)
        return encoder_outputs, x
    
class decoder(nn.Module):
    def __init__(self, channel_list):
        super().__init__()
        self.upsamples = nn.ModuleList([
            nn.ConvTranspose2d(channel_list[i], channel_list[i+1], 2, 2)  
            for i in range(len(channel_list)-1)
        ])
        self.blocks = nn.ModuleList([
            conv_block(channel_list[i], channel_list[i+1])
            for i in range(len(channel_list)-1)
        ])
    
    def forward(self, x, encoder_outputs):
        encoder_outputs_reversed = encoder_outputs[::-1]
        
        for ind, (upsample, block) in enumerate( zip(self.upsamples, self.blocks) ):            
            x = upsample(x) # doubles the spatial dimension and decrease channels
            x = torch.cat([x, encoder_outputs_reversed[ind]], dim=1) # channels are increased again
            x = block(x) # decrease number of channels
        return x

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
            nn.init.normal_(m.weight.data, 0, 0.02)

## Load UNet

In [5]:
OUTPUT_CHANNEL = len(cfg.segmentation_dict['final_output'])
UNet = unet(cfg.UNet['channel_list'], OUTPUT_CHANNEL=OUTPUT_CHANNEL).to(cfg.device)
UNet.load_state_dict(torch.load( '/kaggle/input/unet256clean/UNet256clean.pth', map_location=cfg.device ))
_ = UNet.eval()

In [6]:
patient_list_train = pd.read_parquet('/kaggle/input/patient-list-train/patient_list_train.parquet')
patient_list_train = patient_list_train[::cfg.DS_RATE]

# Mask creation methods

In [52]:
def include_injured_slices(patient_id, series_id, instance_numbers, percentage=0.25):
    # find injured slices of this session
    df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/image_level_labels.csv')
    df_patient = df.loc[(df['patient_id']==patient_id) & (df['series_id']==series_id)]
    if df_patient.shape[0] == 0:
        return instance_numbers
    injured_slices_extravasation = df_patient.loc[df_patient['injury_name']=='Active_Extravasation', 'instance_number'].values.tolist()
    injured_slices_bowel = df_patient.loc[df_patient['injury_name']=='Bowel', 'instance_number'].values.tolist()
    injured_slices = set(injured_slices_extravasation + injured_slices_bowel)    
    injured_slices_first = min(injured_slices)
    injured_slices_last = max(injured_slices)
    
    # check if we have enough injured slices
    N = len(instance_numbers)
    if len( injured_slices.intersection( set(instance_numbers) ) ) > percentage * N:
        return instance_numbers
    
    # otherwise try to interpolate
    instance_numbers = sorted(instance_numbers)
    first_slice = instance_numbers[0]
    last_slice = instance_numbers[-1]
    all_slices = np.arange(first_slice, last_slice, 1)
    
    # find healthy slices
    healthy_slices = all_slices[ (all_slices > injured_slices_last) | (all_slices < injured_slices_first) ]

    # downsample injured slices
    N_injured = int( np.floor( percentage * N ) )
    injured_slices = np.sort( np.array( list( injured_slices ) ) )
    inds = np.floor(np.linspace(0, injured_slices.size-1, N_injured)).astype(int).flatten()
    injured_slices_downsampled = injured_slices[inds]
    
    # downsample healthy slices
    N_healthy = int( np.floor( (1-percentage) * N ) )
    inds = np.floor(np.linspace(0, healthy_slices.size-1, N_healthy)).astype(int).flatten()
    healthy_slices_downsampled = healthy_slices[inds]
    
    # concatenate healthy and injured
    out = list( np.sort(np.concatenate([healthy_slices_downsampled, injured_slices_downsampled])) )
    return out

In [125]:
def prepare_masks(this_patient_all_slices, model, mode='train'):
    patient_id = this_patient_all_slices['patient_id'].unique()[0]
    series_id = this_patient_all_slices['series_id'].unique()[0]
    serie_path = os.path.join(cfg.data_path[mode], str(patient_id), str(series_id))
    
    # Extract instance numbers 
    instance_numbers = this_patient_all_slices['instance_number'].values.tolist()
    instance_numbers = include_injured_slices(patient_id, series_id, instance_numbers, percentage=0.25)
    instance_numbers = correct_z_orientation(serie_path, instance_numbers)
    
    # correct for numbers of slices with interpolation (create a mock 3d image so you can use f.interpolate)
    x = torch.tensor(torch.tensor(instance_numbers).reshape(-1, 1, 1)).repeat(1, 5, 5).type(torch.float32)
    x_interp = correct_num_slices(x, cfg.num_slices)
    instance_numbers = [int(num) for num in x_interp[: , 0, 0].flatten()]
    
    # create images and concatenate slices
    UNet_inputs = []
    image_origs = []
    for instance_number in instance_numbers:
        instance_number = str(instance_number)

        # -------------------------------------------------- load CT data and transform
        filename = str(instance_number) + '.dcm'
        file_path = os.path.join(cfg.data_path[mode], str(patient_id), str(series_id), filename)
        image_orig = load_CT_slice(file_path)
        if np.count_nonzero(image_orig) == 0:
            print("skipped")
            continue
        # _______________________ transform masks
        image_orig_basic = cfg.transform_dict['basic'](image=image_orig)['image']
        image_orig_norm = cfg.transform_dict['normalize'](image=image_orig_basic)['image']
        image_origs.append(torch.tensor(image_orig_norm))
        UNet_input = T.ToTensor()(image_orig_norm)
        UNet_inputs.append(UNet_input)
        
        # -------------------------------------------------- Pass normalized image through UNet
    UNet_inputs = torch.stack(UNet_inputs, dim=0)
    image_origs = torch.stack(image_origs, dim=0)
    seg_probs = model(UNet_inputs.to(cfg.device)).detach().cpu()
    seg_probs = torch.argmax(seg_probs, dim=1).type(torch.float32)
    # -------------------------------------------------- extract each organ's segmentation
    segs = {}

    for i, instance_number in enumerate(instance_numbers):
        slices = {key: None for key in cfg.segmentation_dict['final_output']}
        for organ in slices.keys():
            organ_index = cfg.segmentation_dict['final_output'].index(organ)
            this_organ_mask = np.where(seg_probs[i, ...] == organ_index, image_origs[i, ...], image_origs[i, ...].min()-1)
            this_organ_mask = T.ToTensor()(torch.tensor(this_organ_mask))
            slices[organ] = this_organ_mask
        
        segs[str(instance_number)] = slices
    del image_origs, UNet_inputs
    
    return segs
        
def correct_num_slices(x, num_slices):
    x = x.unsqueeze(dim=0)
    x = x.permute(0, 2, 1, 3)
    out = f.interpolate(x, [num_slices, x.shape[-1]], mode='nearest')
    out = out.permute(0, 2, 1, 3).squeeze(dim=0)
    return out.clone()

def correct_z_orientation(serie_path, input_data):    
    z_dir = get_z_acquisition_direction( serie_path )
    if z_dir == 'upward':
        return input_data[::-1]
    else:
        return input_data
    
def correct_extravasation_background(img):
    img = np.where(img == img.min(), np.nan, img)
    img_minimum = np.nanmin(img)
    img = np.where(np.isnan(img), img_minimum, img)
    img = cv2.normalize(img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return img

In [130]:
save_dir = Path("/kaggle/working/unet256clean_masks/")
save_dir.mkdir(exist_ok=True)
check_dir = Path("/kaggle/input/unet256clean-masks/unet256clean_masks")
all_patients = patient_list_train.patient_id.unique()
for patient in tqdm(all_patients):
    #try:
    patient_dir = save_dir/str(patient)
    if not (check_dir/str(patient)).exists():
        patient_dir.mkdir(exist_ok=True)
        this_patient_all_slices = patient_list_train[patient_list_train.patient_id==patient]
        series_id_all = this_patient_all_slices['series_id'].unique().tolist()

        if len(series_id_all) == 1:
            masks = {series_id_all[0]: prepare_masks(this_patient_all_slices, UNet)}

        else:
            masks = {}
            aortic_hu_values = this_patient_all_slices.aortic_hu_values.values[0]

            if len(aortic_hu_values) == 1:
                this_patient_all_slices_this_session = this_patient_all_slices[this_patient_all_slices.series_id == series_id_all[0]]
                masks[series_id_all[0]] = prepare_masks(this_patient_all_slices_this_session, UNet)

            else:
                for series_id in series_id_all:
                    this_patient_all_slices_this_session = this_patient_all_slices[this_patient_all_slices.series_id == series_id]

                    if this_patient_all_slices_this_session.aortic_hu.values[0] == np.max(this_patient_all_slices_this_session.aortic_hu_values.values[0]):
                        series_masks = prepare_masks(this_patient_all_slices_this_session, UNet)

                        background_masks = {}
                        for instance, mask in series_masks.items():
                            background_masks[instance] = {"Background": mask["Background"]}
                        masks[series_id] = background_masks

                    elif this_patient_all_slices_this_session.aortic_hu.values[0] == np.min(this_patient_all_slices_this_session.aortic_hu_values.values[0]):
                        masks[series_id] = prepare_masks(this_patient_all_slices_this_session, UNet)

                    else:
                        continue

        for series_id, series_mask in masks.items():
            series_dir = patient_dir/str(series_id)
            series_dir.mkdir(exist_ok=True)

            for fname, mask in series_mask.items():
                for organ, tensor in mask.items():
                    norm_image = cv2.normalize(tensor.numpy(), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8U)             
                    if organ =="Background":
                        norm_image = correct_extravasation_background(norm_image)

                    if (np.count_nonzero(norm_image) < 40) and (np.count_nonzero(norm_image) > 0):
                        norm_image = np.zeros_like(norm_image)
                    if np.count_nonzero(norm_image) != 0:
                        cv2.imwrite(str(series_dir/(f"{fname}_{organ}.png")), norm_image)
                            
  #  except Exception as error:
#     print(f"{patient} failed.")
#        print(error)

100%|██████████| 1/1 [00:07<00:00,  7.99s/it]


In [131]:
!tar -czf /kaggle/working/output.tar.gz /kaggle/working/unet256clean_masks/

from IPython.display import FileLink

FileLink(r'/kaggle/working/output.tar.gz')

tar: Removing leading `/' from member names


/kaggle/working/output.tar.gz